In [14]:
import pandas as pd
import numpy as np

In [15]:
df = pd.read_csv("start_up_raw.csv")
df.head(20)
pd.set_option('display.max_rows',1111)

In [18]:
# we remove rows thats missing funding amount
df = df[df['funding_total_usd'] != "-"]

# remove wrong rows
df=df[df['founded_at']<=df['first_funding_at']]

# there're rows without fouded_at date
# check if there's data on 1900-01-01 so we can save the datapoints and filter out later if timeline analysis needed
df[df['founded_at']=='1900-01-01']

,permalink,name,homepage_url,category_list,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,first_funding_at,last_funding_at


In [19]:


# we fill missing foundation date with 1900 to filter them out later
df['founded_at'].fillna('1900-1-1', inplace=True)

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38954 entries, 3 to 66366
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   permalink          38954 non-null  object
 1   name               38953 non-null  object
 2   homepage_url       37348 non-null  object
 3   category_list      38606 non-null  object
 4   funding_total_usd  38954 non-null  object
 5   status             38954 non-null  object
 6   country_code       36631 non-null  object
 7   state_code         35831 non-null  object
 8   region             36150 non-null  object
 9   city               36151 non-null  object
 10  funding_rounds     38954 non-null  int64 
 11  founded_at         38954 non-null  object
 12  first_funding_at   38954 non-null  object
 13  last_funding_at    38954 non-null  object
dtypes: int64(1), object(13)
memory usage: 4.5+ MB


In [21]:
# we drop the null rows and reset the index
df = df.dropna().reset_index()

df.info() # after cleaning we have 44618 rows

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34072 entries, 0 to 34071
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   index              34072 non-null  int64 
 1   permalink          34072 non-null  object
 2   name               34072 non-null  object
 3   homepage_url       34072 non-null  object
 4   category_list      34072 non-null  object
 5   funding_total_usd  34072 non-null  object
 6   status             34072 non-null  object
 7   country_code       34072 non-null  object
 8   state_code         34072 non-null  object
 9   region             34072 non-null  object
 10  city               34072 non-null  object
 11  funding_rounds     34072 non-null  int64 
 12  founded_at         34072 non-null  object
 13  first_funding_at   34072 non-null  object
 14  last_funding_at    34072 non-null  object
dtypes: int64(2), object(13)
memory usage: 3.9+ MB


In [22]:
##### treat category######

# there are many category tags, we only take 4 of them
df[['category','sub_category_1','sub_category_2','sub_category_3']] = df['category_list'].str.split('|', 3, expand=True)

# make a copy of categoty
df['label'] = df['category']

### group categories to labels
df.loc[df['category'].astype(str).str.contains('Software|developer|saas|search', case = False),'label'] = 'Software'
df.loc[df['category'].astype(str).str.contains('Ecommerce|E-comm|E-com|e-book|online|b2b', case=False), 'label'] = 'E-commerce'
df.loc[df['category'].astype(str).str.contains('Web|Website|cyber|browser|internet', case=False), 'label'] = 'Web'
df.loc[df['category'].astype(str).str.contains('Mobile|Android|ipad|iphone|Messaging|Chat|android', case=False),'label'] = 'Mobile'
df.loc[df['category'].astype(str).str.contains('Analytics|Analytic|Analysing|Analytical|consulting|business', case=False),'label'] = 'Business Analytics'
df.loc[df['category'].astype(str).str.contains('App|Apps|Applications|Application', case=False),'label'] = 'Apps'
df.loc[df['category'].astype(str).str.contains('Health|Medical|Medicine|Fitness|Pharm|clinical|doctor|dental|Diagnostic|hospital', case=False),'label'] = 'Health'
df.loc[df['category'].astype(str).str.contains('Clean|Green|Renewable|Energy|environment|gas', case=False),'label'] = 'Energy'
df.loc[df['category'].astype(str).str.contains('Games|Game|Gaming', case=False),'label'] = 'Games'
df.loc[df['category'].astype(str).str.contains('Finance|Financial|Financing|Accounting|Investment|Credit|Banking|crowd|fintech|payment|insurance|billing|trading', case=False),'label'] = 'Finance'
df.loc[df['category'].astype(str).str.contains('Education|Educational|Edtech|colleges', case=False),'label'] = 'Education'
df.loc[df['category'].astype(str).str.contains('Hotel|Hospitablity | Resort', case=False),'label'] = 'Hospitality'
df.loc[df['category'].astype(str).str.contains('Realtor|Real|property', case=False),'label'] = 'Real Estate'
df.loc[df['category'].astype(str).str.contains('Social Media|media|music|news|content|journal|audio|broadcasting|blogging', case=False),'label'] = 'Media/Content'
df.loc[df['category'].astype(str).str.contains('marketing|branding|Advertising|Ads|Adtech|market|ad targeting', case=False),'label'] = 'Marketing'
df.loc[df['category'].astype(str).str.contains('Cars|Automotive|Vehicle|Bus', case=False),'label'] = 'Automotive'
df.loc[df['category'].astype(str).str.contains('semiconductor', case=False),'label'] = 'Semiconductor'
df.loc[df['category'].astype(str).str.contains('aerospace|astro|drone', case=False),'label'] = 'Aerospace'
df.loc[df['category'].astype(str).str.contains('chemic', case=False),'label'] = 'Chemicals'
df.loc[df['category'].astype(str).str.contains('manufac', case=False),'label'] = 'Manufacturing'
df.loc[df['category'].astype(str).str.contains('lifestyle|diy|home renovation|photo|cosmetics', case=False),'label'] = 'Lifestyle'
df.loc[df['category'].astype(str).str.contains('transport', case=False),'label'] = 'Transportation'
df.loc[df['category'].astype(str).str.contains('legal|law', case=False),'label'] = 'Legal'
df.loc[df['category'].astype(str).str.contains('logistics|delivery', case=False),'label'] = 'Logistics'
df.loc[df['category'].astype(str).str.contains('agri', case=False),'label'] = 'Agriculture'
df.loc[df['category'].astype(str).str.contains('Consumer|retail|babies|pets|coupons|food|coffee|beer|groceries', case=False),'label'] = 'Consumers'
df.loc[df['category'].astype(str).str.contains('Enterprise|Entrepreneur|corporate|document|human resource|nonprofit|career|employ|crm|recruit|email|collaboration', case=False),'label'] = 'Enterprise'
df.loc[df['category'].astype(str).str.contains('Technology|bid data|data|hardware|cloud|artificial intel|tech|3D|electronic|bitcoin|automation|algorithms|machine learning|robot', case=False),'label'] = 'Technology'
df.loc[df['category'].astype(str).str.contains('fashion|concerts|design|art|beauty|tourism|sports|restaurant|leisure|travel|cooking|film|entertain|video', case=False),'label'] = 'Leisure'

# those cant group go to others
list = ['Software','E-commerce','Web', 'Mobile', 'Business Analytics','Transportation','Apps','Health','Energy','Games','Finance','Education','Hospitality','Real Estate','Media/Content','Manufacturing', 'Chemicals','Semiconductor', 'Aerospace','Marketing','Automotive','Consumers', 'Agriculture','Logistics', 'Legal','Enterprise', 'Technology','Leisure','Lifestyle']
df.loc[~df['label'].astype(str).isin(list),'label']='Others'


In [23]:
# group labels into industries
df.loc[df['label'].astype(str).str.contains('transportation|logistics|legal|agriculture|communities|public relation|energy|security|health|education', case=False),'industry'] = 'Public'
df.loc[df['label'].astype(str).str.contains('manufacture|aerospace|manufacturing|chemicals|semiconductor|automotive', case=False),'industry'] = 'Manufacture'
df.loc[df['label'].astype(str).str.contains('software|apps|web|technology|mobile|e-commerce', case=False),'industry'] = 'Tech'
df.loc[df['label'].astype(str).str.contains('finance|business|entreprice|enterprise', case=False),'industry'] = 'Finance/Business'
df.loc[df['label'].astype(str).str.contains('marketing|media|communication', case=False),'industry'] = 'Communication'
df.loc[df['label'].astype(str).str.contains('leisure|game|lifestyle|consumer', case=False),'industry'] = 'Lifestyles'
df.loc[df['label'].astype(str).str.contains('hospitality|real estate', case=False),'industry'] = 'Real Estate'
df.loc[df['label'].astype(str).str.contains('others', case=False),'industry'] = 'Others'

# check if there's rows uncategorized
df[df['industry'].isnull()]

,index,permalink,name,homepage_url,category_list,funding_total_usd,status,country_code,state_code,region,...,funding_rounds,founded_at,first_funding_at,last_funding_at,category,sub_category_1,sub_category_2,sub_category_3,label,industry


In [13]:
# pick relevant cols
df_final = df[['name', 'funding_total_usd', 'status', 'country_code', 'state_code', 'region',
       'city', 'funding_rounds', 'founded_at', 'first_funding_at',
       'last_funding_at', 'category', 'label', 'industry']]

# export to csv
df_final.to_csv('startup_clean.csv')